# Clasificación de imágenes con ImageNet

La lista de objetos que se pueden clasificar mediante las redes preentrenadas se encuentra en este [enlace](https://github.com/dusty-nv/jetson-inference/blob/master/data/networks/ilsvrc12_synset_words.txt)

--------------

## Uso de los programas precompilados de la Jetson

Aquí se pueden encontrar los códigos de los programas precompilados:
 * [C++](https://github.com/dusty-nv/jetson-inference/blob/master/examples/imagenet/imagenet.cpp)
 * [Python](https://github.com/dusty-nv/jetson-inference/blob/master/python/examples/imagenet.py)

Ir a la carpeta con los programas precompilados con el siguiente comando

```
$ cd jetson-inference/build/aarch64/bin
```

A continuación, clasifiquemos una imagen de ejemplo con el programap precompilado imagenet, tanto en C++ como en Python. Si está utilizando el contenedor Docker, es recomendable guardar la imagen de salida clasificada en el directorio images/test. Estas imágenes se podrán ver fácilmente desde su dispositivo host en el directorio jetson-inference/data/images/test.

```
# C++
$ ./imagenet images/orange_0.jpg images/test/orange_0.jpg     # (default network is googlenet)

# Python
$ ./imagenet.py images/orange_0.jpg images/test/orange_0.jpg  # (default network is googlenet)
```

Si queremos clasificar varias imágenes

```
# C++
$ ./imagenet "images/object_*.jpg" "images/test/object_%i.jpg"     # (default network is googlenet)

# Python
$ ./imagenet.py "images/object_*.jpg" "images/test/object_%i.jpg"  # (default network is googlenet)
```

 > **nota**: cuando se usen asteriscos, hay que escribirlos siempre entre comillas ("*.jpg"). De lo contrario, el sistema operativo expandirá automáticamente la secuencia y modificará el orden de los argumentos en la línea de comandos, lo que puede resultar en que una de las imágenes de entrada sea sobrescrita por la salida.

Si queremos usar otras redes (por defecto se usa GoogleNet) hay que usar el flag ```--network```

```
# C++
$ ./imagenet --network=resnet-18 "images/cat_*.jpg" "images/test/cat_%i.jpg"     # (default network is googlenet)

# Python
$ ./imagenet.py --network=resnet-18 "images/cat_*.jpg" "images/test/cat_%i.jpg"  # (default network is googlenet)
```

Las redes que podemos usar para clasificación son:
|Network|argumento CLI|Network Type enum|
|------|--------------|-----------------|
|AlexNet|alexnet|ALEXNET|
|GoogleNet|googlenet|GOOGLENET|
|GoogleNet-12|googlenet-12|GOOGLENET_12|
|ResNet-18|resnet-18|RESNET_18|
|ResNet-50|resnet-50|RESNET_50|
|ResNet-101|resnet-101|RESNET_101|
|ResNet-152|resnet-152|RESNET_152|
|VGG-16|vgg-16|VGG-16|
|VGG-19|vgg-19|VGG-19|
|Inception-v4|inception-v4|INCEPTION_V4|

Si se quiere procesar un videdo solo hay que indicarlo en la entrada

Primero lo descargamos

```
# Download test video (thanks to jell.yfish.us)
$ wget https://nvidia.box.com/shared/static/tlswont1jnyu3ix2tbf7utaekpzcx4rc.mkv -O jellyfish.mkv
```

Y ya lo podemos procesar

```
# C++
$ ./imagenet --network=resnet-18 jellyfish.mkv images/test/jellyfish_resnet18.mkv

# Python
$ ./imagenet.py --network=resnet-18 jellyfish.mkv images/test/jellyfish_resnet18.mkv
```

--------------

## Crear un programa de clasificación en Python

Como vamos a crear un programa, lo primero que tenemos que hacer es crear una carpeta en el Host donde guardaremos el programa

```
$ cd ~/
$ mkdir my-recognition-python
$ cd my-recognition-python
$ touch my-recognition.py
$ chmod +x my-recognition.py
```

Nos descargamos unas imágenes de osos para probar

```
$ wget https://github.com/dusty-nv/jetson-inference/raw/master/data/images/black_bear.jpg 
$ wget https://github.com/dusty-nv/jetson-inference/raw/master/data/images/brown_bear.jpg
$ wget https://github.com/dusty-nv/jetson-inference/raw/master/data/images/polar_bear.jpg
```

A continuación lo que hay que hacer es lanzar el Docker con una carpeta del Host compartida, para que así cuando se cierre el Docker no se borre el programa, para ello lanzamos el Docker con el siguiente comando

```
$ docker/run.sh --volume ~/my-recognition-python:/my-recognition-python   # mounted inside the container to /my-recognition-python 
```

Una vez dentro del Docker ir a la carpeta con los siguientes comandos

```
$ cd ../
$ cd my-recognition-python
```

Editar el archivo .py con el siguiente comando

```
$ nano my-recognition.py
```

Para crear un programa como el precompilado escribimos el siguiente código

```Python
import jetson.inference
import jetson.utils

import argparse
import sys


# parse the command line
parser = argparse.ArgumentParser(description="Classify a live camera stream using an image recognition DNN.", 
                                 formatter_class=argparse.RawTextHelpFormatter, epilog=jetson.inference.imageNet.Usage() +
                                 jetson.utils.videoSource.Usage() + jetson.utils.videoOutput.Usage() + jetson.utils.logUsage())

parser.add_argument("input_URI", type=str, default="", nargs='?', help="URI of the input stream")
parser.add_argument("output_URI", type=str, default="", nargs='?', help="URI of the output stream")
parser.add_argument("--network", type=str, default="googlenet", help="pre-trained model to load (see below for options)")
parser.add_argument("--camera", type=str, default="0", help="index of the MIPI CSI camera to use (e.g. CSI camera 0)\nor for VL42 cameras, the /dev/video device to use.\nby default, MIPI CSI camera 0 will be used.")
parser.add_argument("--width", type=int, default=1280, help="desired width of camera stream (default is 1280 pixels)")
parser.add_argument("--height", type=int, default=720, help="desired height of camera stream (default is 720 pixels)")
parser.add_argument('--headless', action='store_true', default=(), help="run without display")

is_headless = ["--headless"] if sys.argv[0].find('console.py') != -1 else [""]

try:
	opt = parser.parse_known_args()[0]
except:
	print("")
	parser.print_help()
	sys.exit(0)


# load the recognition network
net = jetson.inference.imageNet(opt.network, sys.argv)

# create video sources & outputs
input = jetson.utils.videoSource(opt.input_URI, argv=sys.argv)
output = jetson.utils.videoOutput(opt.output_URI, argv=sys.argv+is_headless)
font = jetson.utils.cudaFont()

# process frames until the user exits
while True:
	# capture the next image
	img = input.Capture()

	# classify the image
	class_id, confidence = net.Classify(img)

	# find the object description
	class_desc = net.GetClassDesc(class_id)

	# overlay the result on the image	
	font.OverlayText(img, img.width, img.height, "{:05.2f}% {:s}, {:s} | Network {:.0f} FPS".format(confidence * 100, class_desc, net.GetNetworkName(), net.GetNetworkFPS()), 5, 5, font.White, font.Gray40)
	
	# render the image
	output.Render(img)

	# update the title bar
	output.SetStatus("{:s} | Network {:.0f} FPS".format(net.GetNetworkName(), net.GetNetworkFPS()))

	# print out performance info
	net.PrintProfilerTimes()

	# exit on input/output EOS
	if not input.IsStreaming() or not output.IsStreaming():
		break

```

Ejecutar el programa con el siguiente comando

```
$ python3 my-recognition.py /dev/video0
```

En este caso abrirá la webcam, se pueden introducir las mismas variables que con el programa precompilado

--------------

## Crear un programa ded clasificación en C++

Como vamos a crear un programa, lo primero que tenemos que hacer es crear una carpeta en el Host donde guardaremos el programa

```
$ cd ~/
$ mkdir my-recognition-cpp
$ cd my-recognition-cpp
$ touch my-recognition.cpp
$ chmod +x my-recognition.cpp
$ touch CMakeLists.txt
$ chmod +x CMakeLists.txt
```

Nos descargamos unas imágenes de osos para probar

```
$ wget https://github.com/dusty-nv/jetson-inference/raw/master/data/images/black_bear.jpg 
$ wget https://github.com/dusty-nv/jetson-inference/raw/master/data/images/brown_bear.jpg
$ wget https://github.com/dusty-nv/jetson-inference/raw/master/data/images/polar_bear.jpg
```

A continuación lo que hay que hacer es lanzar el Docker con una carpeta del Host compartida, para que así cuando se cierre el Docker no se borre el programa, para ello lanzamos el Docker con el siguiente comando

```
$ docker/run.sh --volume ~/my-recognition-cpp:/my-recognition-cpp   # mounted inside the container to /my-recognition-cpp
```

Una vez dentro del Docker ir a la carpeta con los siguientes comandos

```
$ cd ../
$ cd my-recognition-cpp
```

Editar el archivo .py con el siguiente comando

```
$ nano my-recognition.cpp
```

Para crear un programa como el precompilado escribimos el siguiente código

```C++
#include <jetson-utils/videoSource.h>
#include <jetson-utils/videoOutput.h>

#include <jetson-utils/cudaFont.h>
#include <jetson-inference/imageNet.h>

#include <signal.h>


#ifdef HEADLESS
	#define IS_HEADLESS() "headless"	// run without display
#else
	#define IS_HEADLESS() (const char*)NULL
#endif


bool signal_recieved = false;

void sig_handler(int signo)
{
	if( signo == SIGINT )
	{
		LogVerbose("received SIGINT\n");
		signal_recieved = true;
	}
}

int usage()
{
	printf("usage: imagenet [--help] [--network=NETWORK] ...\n");
	printf("                input_URI [output_URI]\n\n");
	printf("Classify a video/image stream using an image recognition DNN.\n");
	printf("See below for additional arguments that may not be shown above.\n\n");	
	printf("positional arguments:\n");
	printf("    input_URI       resource URI of input stream  (see videoSource below)\n");
	printf("    output_URI      resource URI of output stream (see videoOutput below)\n\n");

	printf("%s", imageNet::Usage());
	printf("%s", videoSource::Usage());
	printf("%s", videoOutput::Usage());
	printf("%s", Log::Usage());

	return 0;
}

int main( int argc, char** argv )
{
	/*
	 * parse command line
	 */
	commandLine cmdLine(argc, argv, IS_HEADLESS());

	if( cmdLine.GetFlag("help") )
		return usage();


	/*
	 * attach signal handler
	 */
	if( signal(SIGINT, sig_handler) == SIG_ERR )
		LogError("can't catch SIGINT\n");


	/*
	 * create input stream
	 */
	videoSource* input = videoSource::Create(cmdLine, ARG_POSITION(0));

	if( !input )
	{
		LogError("imagenet:  failed to create input stream\n");
		return 0;
	}


	/*
	 * create output stream
	 */
	videoOutput* output = videoOutput::Create(cmdLine, ARG_POSITION(1));
	
	if( !output )
		LogError("imagenet:  failed to create output stream\n");	
	

	/*
	 * create font for image overlay
	 */
	cudaFont* font = cudaFont::Create();
	
	if( !font )
	{
		LogError("imagenet:  failed to load font for overlay\n");
		return 0;
	}


	/*
	 * create recognition network
	 */
	imageNet* net = imageNet::Create(cmdLine);
	
	if( !net )
	{
		LogError("imagenet:  failed to initialize imageNet\n");
		return 0;
	}


	/*
	 * processing loop
	 */
	while( !signal_recieved )
	{
		// capture next image image
		uchar3* image = NULL;

		if( !input->Capture(&image, 1000) )
		{
			// check for EOS
			if( !input->IsStreaming() )
				break;

			LogError("imagenet:  failed to capture next frame\n");
			continue;
		}

		// classify image
		float confidence = 0.0f;
		const int img_class = net->Classify(image, input->GetWidth(), input->GetHeight(), &confidence);
	
		if( img_class >= 0 )
		{
			LogVerbose("imagenet:  %2.5f%% class #%i (%s)\n", confidence * 100.0f, img_class, net->GetClassDesc(img_class));	

			if( font != NULL )
			{
				char str[256];
				sprintf(str, "%05.2f%% %s", confidence * 100.0f, net->GetClassDesc(img_class));
	
				font->OverlayText(image, input->GetWidth(), input->GetHeight(),
						        str, 5, 5, make_float4(255, 255, 255, 255), make_float4(0, 0, 0, 100));
			}
		}	

		// render outputs
		if( output != NULL )
		{
			output->Render(image, input->GetWidth(), input->GetHeight());

			// update status bar
			char str[256];
			sprintf(str, "TensorRT %i.%i.%i | %s | Network %.0f FPS", NV_TENSORRT_MAJOR, NV_TENSORRT_MINOR, NV_TENSORRT_PATCH, net->GetNetworkName(), net->GetNetworkFPS());
			output->SetStatus(str);	

			// check if the user quit
			if( !output->IsStreaming() )
				signal_recieved = true;
		}

		// print out timing info
		net->PrintProfilerTimes();
	}
	
	
	/*
	 * destroy resources
	 */
	LogVerbose("imagenet:  shutting down...\n");
	
	SAFE_DELETE(input);
	SAFE_DELETE(output);
	SAFE_DELETE(net);
	
	LogVerbose("imagenet:  shutdown complete.\n");
	return 0;
}

```

Editar el CMakeList.txt con lo siguiente

```
# require CMake 2.8 or greater
cmake_minimum_required(VERSION 2.8)

# declare my-recognition project
project(my-recognition)

# import jetson-inference and jetson-utils packages.
# note that if you didn't do "sudo make install"
# while building jetson-inference, this will error.
find_package(jetson-utils)
find_package(jetson-inference)

# CUDA and Qt4 are required
find_package(CUDA)
find_package(Qt4)

# setup Qt4 for build
include(${QT_USE_FILE})
add_definitions(${QT_DEFINITIONS})

# compile the my-recognition program
cuda_add_executable(my-recognition my-recognition.cpp)

# link my-recognition to jetson-inference library
target_link_libraries(my-recognition jetson-inference)
```

Compilar el código con los siguientes comandos

```
$ cmake .
$ make
```

Ejecutar el programa con el siguiente comando

```
$ ./my-recognition /dev/video0
```

En este caso abrirá la webcam, se pueden introducir las mismas variables que con el programa precompilado